In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1
)

### from_template() 메소드
* 변수를 중괄호로 묶어서 템플릿에 정의

In [4]:
from langchain_core.prompts import PromptTemplate

template = "{name}의 직업은 무엇인가요?"

prompt = PromptTemplate.from_template(template)

print(prompt)

input_variables=['name'] input_types={} partial_variables={} template='{name}의 직업은 무엇인가요?'


In [5]:
prompt = prompt.format(name="bear")

print(prompt)

bear의 직업은 무엇인가요?


In [6]:
from langchain_core.prompts import PromptTemplate

template = "{language}는 누가 만들었나요"

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

In [8]:
# print(chain.invoke({"language" : "Python"}).content)
print(chain.invoke({"Python"}).content)

Python은 귀도 반 로섬(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 이 프로그래밍 언어를 만들 때 코드의 가독성과 사용의 용이성을 중시하였으며, Python은 이후로도 많은 개발자들 사이에서 인기를 끌고 있습니다.


### PromptTemplate 객체 생성과 동시에 prompt 생성

* input_variables 인자를 사용해 변수를 지정한다.
* 템플릿에 작성한 변수가 input_variables에 없으면 예외를 발생시켜준다.

In [10]:
template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["language"],
)

print(prompt)

prompt.format(language="Python")

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


'Python는 누가 만들었나요?'

partial_variables

In [13]:
template = "{language1}과 {language2}는 각각 누가 만들었나요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["language1"],
    partial_variables={
        "language2" : "Java" #dictionary 형태로 partiak_variables 작성
    }
)

print(prompt)

input_variables=['language1'] input_types={} partial_variables={'language2': 'Java'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [14]:
prompt.format(language1="Python")

'Python과 Java는 각각 누가 만들었나요?'

partial()

In [15]:
prompt_partial = prompt.partial(language2="JavaScript")

print(prompt_partial)

input_variables=['language1'] input_types={} partial_variables={'language2': 'JavaScript'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [16]:
prompt_partial.invoke("Python")

StringPromptValue(text='Python과 JavaScript는 각각 누가 만들었나요?')

In [17]:
chain = prompt_partial | llm

print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 이 언어를 만들 때 코드의 가독성과 사용의 용이성을 중시했습니다.

JavaScript는 브렌던 아이크(Brendan Eich)에 의해 1995년에 개발되었습니다. 처음에는 웹 브라우저에서 동적인 기능을 추가하기 위해 만들어졌으며, 이후 웹 개발에서 중요한 역할을 하게 되었습니다.


파일로부터 template 읽어오기

In [24]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/language_simple.yaml", encoding="UTF-8")

print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language} 언어에 대해 3줄로 설명해줘.'


In [25]:
prompt.format(language="Python")

'Python 언어에 대해 3줄로 설명해줘.'

In [26]:
prompt2 = load_prompt("prompts/language.yaml", encoding="UTF-8")

print(prompt2.format(language="Python"))

Python 언어에 대해 설명해주세요.
언어의 특징을 다음의 양식에 맞게 정리하세요.
300자 내외로 작성하세요.
한글로 작성하세요.
---
#양식
1. 특징
2. 제작자
3. 대표적인 프레임워크
4. 많이 사용되는 분야



In [28]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name="gpt-4o-mini", template=0.0) | StrOutputParser()

answer = chain.invoke("Python")

print(answer)

c:\Users\20112\miniforge3\envs\langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3517: UserWarning: WARNING! template is not default parameter.
                template was transferred to model_kwargs.
                Please confirm that template is what you intended.
  if await self.run_code(code, result, async_=asy):


TypeError: Completions.create() got an unexpected keyword argument 'template'

ChatPromptTemplate

In [29]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{language}의 제작자는 누구인가요?")

print(chat_prompt)

input_variables=['language'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='{language}의 제작자는 누구인가요?'), additional_kwargs={})]


In [31]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 인공지능 어시스턴트입니다. 당신의 이름은{name}입니다"),
        ("human","반가워요!"),
        ("ai", "안녕하세요 무엇을 도와드릴까요"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    name="구찌", user_input="당신의 이름은 무엇입니까"
)

print(messages)

[SystemMessage(content='당신은 친절한 인공지능 어시스턴트입니다. 당신의 이름은구찌입니다', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요 무엇을 도와드릴까요', additional_kwargs={}, response_metadata={}), HumanMessage(content='당신의 이름은 무엇입니까', additional_kwargs={}, response_metadata={})]


In [32]:
print(llm.invoke(messages).content)

제 이름은 구찌입니다! 당신과 대화하게 되어 기쁩니다. 어떤 도움이 필요하신가요?


MessagePlaceHolder

In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
     (
         "system",
         "당신은 요약 전문 AI 어시스턴트입니다 당신의 임무는 키워드로 대화를 요약하는 것입니다"
     ),
     MessagesPlaceholder(variable_name="conversation"),
     ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다")
    ]
)

print(chat_prompt)

input_variables=['conversation', 'word_count'] input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.m

In [34]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
    ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
    ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
    ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
    ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")

]
)
print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다 당신의 임무는 키워드로 대화를 요약하는 것입니다
Human: 파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?
AI: 아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다.
Human: set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ
AI: 네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊
Human: 지금까지의 대화를 5 단어로 요약합니다


In [35]:
chain = chat_prompt | llm | StrOutputParser()

In [36]:
input = {
    "word_count":5,
    "conversation":[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
    ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
    ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
    ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
}

chain.invoke(input)

'파이썬, 리스트, 중복 제거, set.'

In [37]:
!pip install langchain-community

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ----------------- ---------------------- 1.0/2.4 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 4.7 MB/s eta 0:00:00


In [39]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma2:9b")

response = llm.invoke("지구의 자전 주기는?")

response

'지구의 자전 주기는 약 **24시간** 입니다. \n\n이를 "하루"라고 부르며, 지구가 자신의 축을 한 바퀴 돌아서 하늘에서 보이는 별자리들이 변하는 것을 의미합니다. 😊\n'

In [40]:
from langchain.chat_models import ChatOllama

model = ChatOllama(
    model="gemma2:9b",
    temperature=0.1
)

C:\Users\20112\AppData\Local\Temp\ipykernel_7688\2642841664.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(


In [41]:
chain = chat_prompt | model | StrOutputParser()

chain.invoke(input)

'파이썬, 중복 제거, set 사용, 간단 해결.  \n'